In [27]:
include("../../src/WaterLily.jl")
WaterLily = Main.WaterLily;
using Plots; gr()
using StaticArrays
using JLD

In [28]:
inside(a::AbstractArray) = CartesianIndices(map(ax->first(ax)+1:last(ax)-1,axes(a)))
@inline CI(a...) = CartesianIndex(a...)
"""
    δ(i,N::Int)
    δ(i,I::CartesianIndex{N}) where {N}

Return a CartesianIndex of dimension `N` which is one at index `i` and zero elsewhere.
"""
δ(i,::Val{N}) where N = CI(ntuple(j -> j==i ? 1 : 0, N))
δ(i,I::CartesianIndex{N}) where N = δ(i, Val{N}())

δ (generic function with 2 methods)

In [29]:
function InterpolOmega(xOrig,omega)
    x = xOrig .+ 2.0
    floorx = floor.(x)
    I = CartesianIndex(Int.(floorx)...)
    residual = x.-floorx
    N = length(I)
    s = 0.0
    for i ∈ N
        s += omega[I]*(1-residual[i]) + omega[I+δ(i,I)]*residual[i]
    end
    return s/N
end

InterpolOmega (generic function with 1 method)

In [30]:
function flood(f::Array;shift=(0.,0.),cfill=:RdBu_11,clims=(),levels=10,kv...)
    if length(clims)==2
        @assert clims[1]<clims[2]
        @. f=min(clims[2],max(clims[1],f))
    else
        clims = (minimum(f),maximum(f))
    end
    Plots.contourf(axes(f,1).+shift[1],axes(f,2).+shift[2],f',
        linewidth=0, levels=levels, color=cfill, clims = clims, 
        aspect_ratio=:equal; kv...)
end

addbody(x,y;c=:black) = Plots.plot!(Shape(x,y), c=c, legend=false)
function body_plot!(sim;levels=[0],lines=:black,R=inside(sim.flow.p))
    WaterLily.measure_sdf!(sim.flow.σ,sim.body,WaterLily.time(sim))
    contour!(sim.flow.σ[R]';levels,lines)
end

function sim_gif!(sim;duration=1,step=0.1,verbose=true,R=inside(sim.flow.p),
                    remeasure=false,plotbody=false,kv...)
    t₀ = round(WaterLily.sim_time(sim))
    omegaInter = Float64[]
    @time @gif for tᵢ in range(t₀,t₀+duration;step)
        WaterLily.sim_step!(sim,tᵢ;remeasure)
        for I∈inside(sim.flow.σ)
            sim.flow.σ[I] = WaterLily.curl(3,I,sim.flow.u)*sim.L/sim.U
        end
        WaterLily.BCPer!(sim.flow.σ)
        flood(sim.flow.σ[R]; kv...)
        plotbody && body_plot!(sim)
        verbose && println("tU/L=",round(tᵢ,digits=4),
            ", Δt=",round(sim.flow.Δt[end],digits=3))
        push!(omegaInter,InterpolOmega(SA[mod(16.1+6.4*tᵢ,64),mod(16.1+12.8*tᵢ,64)], sim.flow.σ))
    end
    return omegaInter
end

sim_gif! (generic function with 1 method)

In [31]:
function circle(n,m;Re=250,U=1)
    radius, center = m/8, m/2
    body = WaterLily.AutoBody((x,t)->√sum(abs2, x .- center) - radius)
    return WaterLily.Simulation((n,m), (U,0), radius; ν=U*radius/Re, body)
end

circle (generic function with 1 method)

In [32]:
function TGV(; pow=6, Re=1e5, T=Float32, mem=Array)
    # Define vortex size, velocity, viscosity
    L = 2^pow; U = 1; ν = U*L/Re
    # Taylor-Green-Vortex initial velocity field
    function uλ(i,xyz)
        x,y = @. (xyz-1.5)*2π/L                # scaled coordinates
        i==1 && return -U*(sin(x)*cos(y)+0.2*sin(5x)*cos(5y)) # u_x
        i==2 && return  U*(cos(x)*sin(y)+0.2*cos(5x)*sin(5y)+0.1) # u_y
        return 0.                              # u_z
    end
    # Initialize simulation
    return WaterLily.Simulation((L, L), (0, 0), L; U=U, uλ=uλ, ν=ν, T=T, mem=mem,perdir=(2,))
end

TGV (generic function with 1 method)

In [33]:
function TGVHalfPer(; pow=6, Re=1e5, T=Float32, mem=Array)
    # Define vortex size, velocity, viscosity
    L = 2^pow; U = 1; ν = U*L/Re
    # Taylor-Green-Vortex initial velocity field
    function uλ(i,xyz)
        x,y = @. (xyz-1.5)*2π/L                # scaled coordinates
        i==1 && return U*(sin(x)*sin(y)+0.1sin(10x)*sin(10y)+0.2*rand()+0.0) # u_x
        i==2 && return U*(cos(x)*cos(y)+0.1cos(10x)*cos(10y)+0.2*rand()+0.0) # u_y
        return 0.                              # u_z
    end
    # Initialize simulation
    return WaterLily.Simulation((L, L), (0, 0), L; U=U, uλ=uλ, ν=ν, T=T, mem=mem)
end

TGVHalfPer (generic function with 1 method)

In [34]:
sim = TGV(;Re=10000)
# sim = TGVHalfPer(;Re=10000)

Main.WaterLily.Simulation(1, 64, 1, Main.WaterLily.Flow{2, Float32, Matrix{Float32}, Array{Float32, 3}, Array{Float32, 4}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.18935299 0.18935299 … 0.18935299 0.18935299; 0.0 0.0 … 0.0 0.0;;; -0.08935299 0.1 … -0.08935299 0.1; -0.08935299 0.1 … -0.08935299 0.1; … ; -0.08935299 0.1 … -0.08935299 0.1; -0.08935299 0.1 … -0.08935299 0.1], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.18935299 0.18935299 … 0.18935299 0.18935299; 0.0 0.0 … 0.0 0.0;;; -0.08935299 0.1 … -0.08935299 0.1; -0.08935299 0.1 … -0.08935299 0.1; … ; -0.08935299 0.1 … -0.08935299 0.1; -0.08935299 0.1 … -0.08935299 0.1], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0

In [35]:
omegaInter = sim_gif!(sim,duration=20, step=0.01,clims=(-10,10),plotbody=false,verbose=false,levels=32);


125.369673 seconds (175.19 M allocations: 8.192 GiB, 1.61% gc time)


┌ Info: Saved animation to /tmp/jl_g9gTY1erAl.gif
└ @ Plots /home/tyhuang/.julia/packages/Plots/3BCH5/src/animation.jl:156


In [36]:
#save("omega_Stay.jld", "data", -omegaInter)

In [37]:
omega_Stay = load("omega_Stay.jld")["data"];
omega_Moving = load("omega_Moving.jld")["data"];
t = range(0.00,5,501)

omega_ini = omega_Stay[1];
omega_Ana = omega_ini*exp.(-8*pi^2/100*t);

In [38]:
pythonplot()
plot(t, (omega_Stay[1:end]-omega_Ana[1:end])./omega_Ana[1:end], label="Stationary")
plot!(t, (omega_Moving[1:end]-omega_Ana[1:end])./omega_Ana[1:end],legend=:topleft, label="Traveling")
savefig("TGVHalfPer_TravelingStationaryOmegaError.png")

# plot!()

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/TGVHalfPer_TravelingStationaryOmegaError.png"

In [39]:
pythonplot()
plot(t, omega_Ana[1:end])
savefig("TGVHalfPer_AnalyticalOmega.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/TGVHalfPer_AnalyticalOmega.png"